In [1]:
# Benötigte Pakete:
# !pip install yt-dlp pandas tqdm

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ------------ --------------------------- 1.0/3.2 MB 7.2 MB/s eta 0:00:01
   ----------------------------- ---------- 2.4/3.2 MB 6.4 MB/s eta 0:00:01
   ---------------------------------------- 3.2/3.2 MB 6.0 MB/s eta 0:00:00


In [11]:
import yt_dlp
import csv
import pandas as pd
import os
from tqdm import tqdm
from yt_dlp.utils import DownloadError

In [21]:
df = pd.read_csv("battle_data.csv")
df["upload_date"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 1128 entries, 0 to 1127
Series name: upload_date
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 8.9 KB


In [30]:
# ---- helper: safe atomic write ----
def safe_write_csv_atomic(df, target_path):
    tmp = target_path + '.tmp'
    df.to_csv(tmp, index=False)
    os.replace(tmp, target_path)    # atomarer Austausch

# ---- Playlist-ID-Abfrage (schnell) ----
def fetch_playlist_ids(playlist_url):
    ydl_opts = {'quiet': True, 'extract_flat': True, 'no_warnings': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(playlist_url, download=False)
        return [{'id': v['id'], 'title': v.get('title','')} for v in info['entries']]

# ---- Vollständige Metadaten für ein Video (mit Fehlerbehandlung) ----
def fetch_data_full(video_id):
    ydl_opts = {'quiet': True, 'extract_flat': False, 'no_warnings': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        try:
            info = ydl.extract_info(f"https://www.youtube.com/watch?v={video_id}", download=False)
            return {
                'id': info['id'],
                'title': info.get('title',''),
                'upload_date': info.get('upload_date',''),
                'view_count': info.get('view_count', 0),
                'like_count': info.get('like_count', 0),
                'comment_count': info.get('comment_count', 0),
                'url': f"https://www.youtube.com/watch?v={info['id']}",
                'status': 'active'
            }
        except Exception as e:
            # jede Art von Fehler → markieren, Abruf läuft weiter
            print(f"Warnung: {video_id} konnte nicht abgerufen werden: {e}")
            return {
                'id': video_id,
                'title': '',
                'upload_date': '',
                'view_count': 0,
                'like_count': 0,
                'comment_count': 0,
                'url': f"https://www.youtube.com/watch?v={video_id}",
                'status': 'deleted'
            }

# ---- Robuste Update-Funktion mit Checkpointing / Resume ----
def update_csv_with_checkpoints(playlists, csv_file, checkpoint_every=20):
    """
    - csv_file: finaler Dateiname, z.B. 'battles_data.csv'
    - checkpoint_every: nach wie vielen abgefragten Videos die partial-Datei geschrieben wird
    """
    partial_file = csv_file + '.partial'

    # 1) vorhandene finale CSV laden (falls vorhanden)
    if os.path.exists(csv_file):
        df_existing = pd.read_csv(csv_file)
    else:
        df_existing = pd.DataFrame(columns=['id','title','upload_date','view_count','like_count','comment_count','url','status'])

    # 2) vorhandene partial (Zwischenspeicherung) laden (falls vorhanden)
    if os.path.exists(partial_file):
        df_partial = pd.read_csv(partial_file)
    else:
        df_partial = pd.DataFrame(columns=['id','title','upload_date','view_count','like_count','comment_count','url','status'])

    # 3) bereits bekannte IDs (final + partial) bilden
    existing_ids = set(df_existing['id'].astype(str).tolist()) | set(df_partial['id'].astype(str).tolist())

    # 4) Hauptschleife: durch Playlists, neue IDs ermitteln und nur die neuen komplett abfragen
    for playlist_url in playlists:
        playlist_videos = fetch_playlist_ids(playlist_url)
        new_battles = [v for v in playlist_videos if str(v['id']) not in existing_ids]
        print(f"{len(new_battles)} neue Battles in Playlist gefunden.")

        # fetched_this_run sammelt die in diesem Lauf neu abgefragten Videos
        fetched_this_run = []

        for i, v in enumerate(tqdm(new_battles, desc="Videos abrufen", unit="Video")):
            info = fetch_data_full(v['id'])
            fetched_this_run.append(info)
            existing_ids.add(str(v['id']))

            # periodisch in partial speichern (so gehen Daten bei Abbruch nicht verloren)
            if (i+1) % checkpoint_every == 0:
                # kombiniere vorhandene partial + fetched_this_run und schreibe partial_file
                if df_partial.empty:
                    df_to_save = pd.DataFrame(fetched_this_run)
                else:
                    df_to_save = pd.concat([df_partial, pd.DataFrame(fetched_this_run)], ignore_index=True).drop_duplicates(subset='id')
                # direkt überschreiben (kein atomic rename nötig für partial)
                df_to_save.to_csv(partial_file, index=False)
                print(f"[Checkpoint] Zwischengespeichert nach {i+1} neuen Battles (partial gespeichert).")

        # am Ende der Playlist: die aktuellen gefetchten in df_partial aufnehmen
        if fetched_this_run:
            if df_partial.empty:
                df_partial = pd.DataFrame(fetched_this_run)
            else:
                df_partial = pd.concat([df_partial, pd.DataFrame(fetched_this_run)], ignore_index=True).drop_duplicates(subset='id')

    # 5) Fertig mit allen Playlists: finale Datei atomar schreiben (merge existing + partial)
    if df_partial.empty and df_existing.empty:
        print("Keine Daten gefunden. Keine Datei geschrieben.")
        return

    if df_existing.empty:
        df_final = df_partial
    else:
        df_final = pd.concat([df_existing, df_partial], ignore_index=True).drop_duplicates(subset='id')

    # atomare Speicherung der finalen CSV
    safe_write_csv_atomic(df_final, csv_file)
    print(f"[Fertig] Finale CSV geschrieben: {csv_file} (insg. {len(df_final)} Battles)")

    # 6) partial löschen, weil alles jetzt final ist
    if os.path.exists(partial_file):
        os.remove(partial_file)
        print("Partial-Datei entfernt.")

# ---- Anwendung ----
playlists = [
    "https://www.youtube.com/playlist?list=PLlYYj-TzOlFko09dyz1xsfLtvZdwxKPfK",
    "https://www.youtube.com/playlist?list=PLXpSN5kwAmcgB3KMBUCHtLC-SGwdF2xMa"
    # ggf. weitere Playlists
]

csv_file = "battles_data.csv"
update_csv_with_checkpoints(playlists, csv_file, checkpoint_every=20)


0 neue Videos in Playlist gefunden.


Videos abrufen: 0Video [00:00, ?Video/s]


0 neue Videos in Playlist gefunden.


Videos abrufen: 0Video [00:00, ?Video/s]

[Fertig] Finale CSV geschrieben: battles_data.csv (insg. 1120 Videos)



C:\Users\Rabea\AppData\Local\Temp\ipykernel_7340\3187547126.py:116: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_existing, df_partial], ignore_index=True).drop_duplicates(subset='id')
